In [1]:
from logging import error, exception
# from httplib2.error import FailedToDecompressContent
import pandas as pd
import pygsheets
import os
from datetime import datetime, date
import sys

import gsheet_utility as gsheetUtility
import Email_Code as fkEmail
import data_processing as data_processing
import numpy as np

In [2]:
basePath = '/Users/a/Documents/GitHub/exception_visibility/'
date_master_file = pd.read_csv( '/Users/a/Documents/GitHub/exception_visibility/Dashboard/data/week_details.csv')
date_master_file['weekday'] = pd.to_datetime(date_master_file["weekday"],format = '%d/%m/%Y').dt.date
date_master_file['month_year'] = pd.to_datetime(date_master_file["month_year"],format = '%d/%m/%Y').dt.date

rc_raw_data_location = basePath + 'ETL/RC_Raw_Data/'
mh_raw_data_location = basePath + 'ETL/MH_Raw_Data/'
orphan_raw_data_location = basePath + 'ETL/Orphan_Raw_Data/'
hv_orphan_raw_data_location = basePath + 'ETL/HV_Orphan_Raw_Data/'

today = date.today()
today = str(today.year) + "-" + str('{:02d}'.format(today.month)) + "-" + str('{:02d}'.format(today.day))

rcgsheetdatapath = basePath + 'ETL/gsheet_data/rc_data/' + today + '.csv'
mhgsheetdatapath = basePath + 'ETL/gsheet_data/mh_data/' + today + '.csv'
orphangsheetdatapath = basePath + 'ETL/gsheet_data/orphan_data/' + today + '.csv'
hv_orphangsheetdatapath = basePath + 'ETL/gsheet_data/hv_orphan_data/' + today + '.csv'

error_message = ''
html_message = ''

In [15]:
html_message = html_message + "Data Ingestion Job for High Value Orphan form started at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting High Value data process')
html_message = html_message + "Starting to fetch High Value orphan raw data from google sheet : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'

print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting High Value orphan data process')
hv_orphanData = gsheetUtility.get_gsheet_data('https://docs.google.com/spreadsheets/d/1_lqYbhqhqSuGSxgJoAPYBcBnRtmly91rIznQDqhKKzE/edit?usp=sharing', 'A3', hv_orphangsheetdatapath,'MH High Value Orphans')
hv_orphandbcolumns = ['pv_status','destination_area','cleared_shipment_tracking_id','shipment_value','weeknum','month','date','scanned_timestamp',
                    'zone','motherhub_name','shift_name','shipment_category','no_of_units','orphan_reason','is_invoice_found','received_time','tracking_id','shipment_type','content_details',
                    'lane_details','trip_id','bag_id','orphan_identified_area','orphan_photograph_link','orphan_id','bag_seal_id','brand_name','product_id','price','model',
                    'color','imei1','imei2','serial_number','ram_rom']
# orphanGsheetColumns = orphanData.columns
html_message = html_message + "completed fetching High Value orphan raw data from google sheet : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Fetching High Value orphan data process Completed')
hv_orphanData = gsheetUtility.assignDBColumns(hv_orphanData, gsheet_asset='HV Orphan Data', dbColumns=hv_orphandbcolumns)
hv_orphanData = hv_orphanData.drop(columns=['weeknum','month','date'])
hv_orphanData = data_processing.datatype_conversion(hv_orphanData, 'scanned_timestamp', 'datetime')
hv_orphanData['scanned_date'] = hv_orphanData['scanned_timestamp'].dt.date
hv_orphanData = data_processing.fetch_date_details(hv_orphanData, 'exception_log_timestamp', date_master_file)

hv_orphanData.cleared_shipment_tracking_id = hv_orphanData.cleared_shipment_tracking_id.replace(r'^\s*$', np.NaN, regex=True)
hv_orphanData['is_tracking_id_available'] = ''
hv_orphanData.loc[hv_orphanData.cleared_shipment_tracking_id.isna(), ['is_tracking_id_available']]='No'
hv_orphanData.loc[~hv_orphanData.cleared_shipment_tracking_id.isna(), ['is_tracking_id_available']] = 'Yes'

print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to create orphan raw data files')
html_message = html_message + "Starting to create High Value orphan raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
hv_orphanData['shipment_value'] = hv_orphanData['shipment_value'].str.replace(',', '')
hv_orphanData['shipment_value'] = hv_orphanData['shipment_value'].str.replace('.0', '')
hv_orphanData.shipment_value = hv_orphanData.shipment_value.str.replace(' ', '')
hv_orphanData.shipment_value = hv_orphanData.shipment_value.fillna(0)
hv_orphanData.shipment_value = hv_orphanData[1378:1380].shipment_value.str.replace(r'[^0-9]+', '')
hv_orphanData.shipment_value =pd.to_numeric(hv_orphanData.shipment_value)
hv_orphanData.orphan_id = hv_orphanData.orphan_id.str.upper()
filenames = []
created_raw_files = data_processing.fetch_created_files(hv_orphan_raw_data_location)
filenames = data_processing.create_rewrite_raw_files(hv_orphanData,'scanned_date', created_raw_files, hv_orphan_raw_data_location )
html_message = html_message + "Completed creating High Value orphan raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Completed creating High Value orphan raw data files')



12/22/2021 14:34:33 Starting High Value data process
12/22/2021 14:34:33 Starting High Value orphan data process
12/22/2021, 14:34:33 Starting Fetching Gsheet Data
12/22/2021, 14:34:36 Fetching Gsheet Completed
MH High Value Orphans
A3
12/22/2021 14:34:36 Fetching High Value orphan data process Completed
Converting scanned_timestamp to datetime
Converted scanned_timestamp to datetime
mapping dates and weekend to scanned date
12/22/2021 14:34:36 Starting to create orphan raw data files
fetching names of already created raw files
started creating raw files
creating raw file for the date 2021-07-19.csv
creating raw file for the date 2021-07-22.csv
creating raw file for the date 2021-07-30.csv
creating raw file for the date 2021-07-26.csv
creating raw file for the date 2021-07-25.csv
creating raw file for the date 2021-07-20.csv
creating raw file for the date 2021-07-15.csv
creating raw file for the date 2021-07-24.csv
creating raw file for the date 2021-07-17.csv
creating raw file for the

In [13]:
hv_orphanData[1378:1380].shipment_value.str.replace(r'[^0-9]+', '')

1378     359
1379    9999
Name: shipment_value, dtype: object

In [ ]:
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to collate data for High Value dashboard')
html_message = html_message + "Starting to create High Value orphan raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
orphan_final_columns = ['shipment_value', 'cleared_shipment_tracking_id', 'scanned_timestamp', 'motherhub_name','zone','motherhub_name','shipment_category',
            'orphan_reason','is_invoice_found','received_time','tracking_id','shipment_type','orphan_identified_area',
            'orphan_id', 'scanned_date', 'weekend', 'month', 'year', 'month_year','weeknum','is_tracking_id_available','asset']
created_raw_files = data_processing.fetch_created_files(hv_orphan_raw_data_location)
orphan_dashboard_data = data_processing.collate_data_for_dashboard(datetime.now().date(), 180, created_raw_files, hv_orphan_raw_data_location, orphan_final_columns)
orphan_dashboard_data.to_csv(basePath + 'Dashboard/data/hv_orphan_full_data.csv', index=False)

In [18]:
hv_orphanData

,pv_status,destination_area,cleared_shipment_tracking_id,shipment_value,scanned_timestamp,zone,motherhub_name,shift_name,shipment_category,no_of_units,...,serial_number,ram_rom,asset,scanned_date,weekend,month,year,month_year,weeknum,is_tracking_id_available
0,Pass,return_qc_pass_bulk,NaN,NaN,2021-07-19 00:00:00,North,MotherHub_FRK,Night,Orphan (Non-damage),1,...,EMVCFMROFMUWRWC6,(4/64),HV Orphan Data,2021-07-19,24/07/2021,July,2021,2021-07-01,30,No
1,Pass,return_qc_pass_bulk,NaN,NaN,2021-07-19 00:00:00,North,MotherHub_FRK,Night,Orphan (Non-damage),1,...,CM0094-GK0038951,N/A,HV Orphan Data,2021-07-19,24/07/2021,July,2021,2021-07-01,30,No
2,Fail,fraud_bulk,NaN,NaN,2021-07-19 00:00:00,North,MotherHub_FRK,Night,Orphan (Non-damage),1,...,B801RISO6C00,2/16,HV Orphan Data,2021-07-19,24/07/2021,July,2021,2021-07-01,30,No
3,Pass,return_qc_pass_bulk,NaN,NaN,2021-07-19 00:00:00,North,MotherHub_FRK,Night,Orphan (Non-damage),1,...,J9N5,4/64,HV Orphan Data,2021-07-19,24/07/2021,July,2021,2021-07-01,30,No
4,Pass,return_qc_pass_bulk,NaN,NaN,2021-07-19 00:00:00,North,MotherHub_FRK,Night,Orphan (Non-damage),1,...,A919TBE06C00,8/128,HV Orphan Data,2021-07-19,24/07/2021,July,2021,2021-07-01,30,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,,,NaN,NaN,2021-12-21 19:05:14,North,MotherHub_JKS,Evening,Orphan (Non-damage),1,...,SGJ9DCKFYJ5X4,NA,HV Orphan Data,2021-12-21,25/12/2021,December,2021,2021-12-01,52,No
1478,,,NaN,NaN,2021-12-22 13:00:53,East,MotherHub_HRN,Morning,Orphan (Non-damage),1,...,1914264510BA0DC6,RAM 4 GB & ROM 64 GB,HV Orphan Data,2021-12-22,25/12/2021,December,2021,2021-12-01,52,No
1479,,,NaN,NaN,2021-12-22 13:11:44,East,MotherHub_HRN,Morning,Orphan (Non-damage),1,...,210914812102000218,RAM 4 GB & ROM 64 GB,HV Orphan Data,2021-12-22,25/12/2021,December,2021,2021-12-01,52,No
1480,,,NaN,NaN,2021-12-22 13:17:07,East,MotherHub_HRN,Morning,Orphan (Non-damage),1,...,M6NRCX05S486255,RAM DDR4 8G & ROM 512G PCIE G3 SSD,HV Orphan Data,2021-12-22,25/12/2021,December,2021,2021-12-01,52,No


In [ ]:
df1